In [1]:
# {'sci_fi': [[36, 'NN NN'], [9, 'NN'], [8, 'JJ NN'], [8, 'NR'], [7, 'NR NN'], [6, 'VV NN']], 
#'action': [[24, 'NN NN'], [8, 'VV NN'], [7, 'NN'], [6, 'NR'], [6, 'NN VV'], [5, 'NR NN']], 
#'adventure': [[19, 'NN NN'], [8, 'NR NN'], [5, 'NR'], [4, 'NN'], [3, 'JJ NN'], [3, 'VV NN']], 
#'horror': [[22, 'NN NN'], [11, 'NN'], [8, 'NR'], [7, 'NN VV'], [6, 'VV'], [5, 'VV NN']], 
#'comody': [[13, 'NN NN'], [8, 'JJ NN'], [6, 'NN'], [5, 'VV NN'], [4, 'NR'], [3, 'NN VV']], 
#'romance': [[11, 'NN NN'], [6, 'NR NN'], [4, 'VV NN'], [4, 'NN'], [1, 'NR NR NN'], [1, 'PN NT']]}
# NN NR VV JJ
import pickle
import re
import jieba
from hanziconv import HanziConv
import os
import time
#--------------set variables here--------------------------------------
operationType=3  #1=yield training data 2=yield test data 3= yield generation data
longTrainingInput=5
shortTrainingInput=3
generateInput=3
movieTitlesFile='movie.txt'
#-------------------------------------------------------------------------
#-------------------------------------------------------------------------
#-------------------------------------------------------------------------
#-------------------------------------------------------------------------
#-------------------------------------------------------------------------
#---------------programs starts below-------------------------------------
#-------------------------------------------------------------------------
#---if 1, gets the training movie titles in Eng. and Chi.-----------------
#---if 2 or 3, save the path to Path--------------------------------------
startTime=time.clock()
if operationType==1:
    print("Operation 1; Training; Preparing training data...")
    Path="./input/trainingData/"

    movieTitles=open(Path+movieTitlesFile,'r',encoding='utf-8')
    titlelist=[]
    titleDict={}
    for title in movieTitles.readlines():                       #get movie names in Eng. and Chi. from movie.txt.
        titlelist.append(title.split('\t'))                     #split by tab
    for i in titlelist:
        tmp=i[1].strip("\n")                                    #remove \n in titles
        tmp=HanziConv.toSimplified(tmp)                         #Make it to simplified Chinese
        i[1]=tmp 
    movieTitles.close()

    for t in titlelist:                                         #save the titles to dictionary
        titleDict.update({t[0]:t[1]})
    print("Total movies in movie.txt =  "+str(len(titleDict)))
    file = os.listdir(Path)
    englishname=[]
    for f in file:                                              #Check if there are movie srt not included in movie.txt
        if f.endswith("srt"):
            filename=str(f)
            tmp=filename.strip("srt")
            tmp=tmp.strip(".")
            englishname.append(tmp)
    for name in englishname:
        if name not in titleDict:
            titleDict.update({name:'0'})                       #if not in movie.txt then add movie 
            print(name)                                        #English name:0 in dict. and print the new movie name
    
    movieTitles=open(Path+'movie.txt','w',encoding='utf-8')    
    for k,v in titleDict.items():                              #------delete ':' in chinese title
        temptitle=[]
        for letter in v:
            if "：" in letter:
                pass
            else:
                temptitle.append(letter)
        titleDict.update({k:"".join(temptitle)})                 
        movieTitles.write(k+"\t"+v+"\n")                       #write the new dictionary to movie.txt
    movieTitles.close()
    print("After Checking, total training .srt in folder = "+str(len(file)-1)) #-1 tp exclude movie.txt
    print("Preparation done!")
elif operationType==2:
    print("Operation 2; Testing")
    Path="./input/testData/"
    print("Path saved for Testing Data")
elif operationType==3:
    print("Operation 3; Generate")
    Path="./input/forGeneration/"
    print("Path saved for Title Generation")
endtime=time.clock()
print("Time Spent = "+ str(endtime-startTime)+" sec.")

Operation 3; Generate
Path saved for Title Generation
Time Spent = 0.0004770000000000607 sec.


In [2]:
import pickle
import re
import jieba
from hanziconv import HanziConv
import os
#------------------------------Cleanup srt and get only chinese characters--------------------------------------
#------------------------------Segment the words through Jieba--------------------------------------------------
#------------------------------All words per movie saved to AllInput is for Jaccard similarity use later--------
#------------------------------VectorInput is for the vector calculation later----------------------------------
#------------------------------This is to prepare all the words for word2Vector calculation---------------------
#------------------------------And words for jaccard similarity-------------------------------------------------
startTime=time.clock()
getPath=Path
files = os.listdir(getPath)                              #get files from the directory depends on the operation
AllInput=[]
VectorInput=[]
movieOrder=[]
print("Start Processing .srt files...")
print("Operation = "+str(operationType))
for f in files:
    if f.endswith("txt"):                                #skip txt file
        pass
    else:
        file=str(f)                                      #only process srt file
        #print(file)
        tmp=file.strip("srt")
        tmp=tmp.strip(".")
        movieOrder.append(tmp)
        testSubs=open(getPath+file, 'r',encoding='utf-8')
        string=""
        tmp=[]
        temp=[]
        alll=[]
        for i in testSubs.readlines():                  #append each line to tmp until \n found
            if i == '\n':
                if tmp==[''] or tmp==[]:
                    pass
                else:
                    temp.append(tmp)                   #if \n found, save tmp to temp 
                    tmp=[]                             #then empty tmp
            else:
                tmp.append(i)
        tmp=[]
        for line in temp: 
            tmp=[]                                              #get only chinese characters from the subtitles
            for n in re.findall('[\u4e00-\u9fff]',str(line)):   #This removes the time stamp, 
                tmp.append(n)                                   #and other unnecessary symbols
            string=''.join(tmp)
            if string=='':
                pass
            else:
                alll.append(string)                             #append each cleaned Chinese lines to alll list
            string=""
            
        tmpSeg=[]
        newAllLines=[]
        forVectorLines=[]
        for line in alll:                                      #segment each line in alll list by jieba
            str_in = str(line)
            seg_list = jieba.cut_for_search("".join(HanziConv.toSimplified(str_in)))  
            for i in seg_list:
                newAllLines.append(i)                         #new AllLines stores segmented sentences
                forVectorLines.append(i)
            forVectorLines.append("。")
        toSimp=[]
        for word in newAllLines:
            toSimp.append(word)
        AllInput.append(toSimp)
        toSimp=[]
        for word in forVectorLines:                          
            toSimp.append(word)
        VectorInput.append(toSimp)
print("Total Moive number = "+str(len(AllInput)))
midTime=time.clock()
print("Time Spent Processing Srt. = "+ str(midTime-startTime)+" sec.")
if operationType==1:                                        #if operation is training,
    print("Saving sentences to vectorInput.txt...")         #save sentences for calculatiing vector later
    outfile=open("vectorInput.txt","w",encoding="utf-8")    #to file: vectorInput
    for i in VectorInput:
        for word in i:
            outfile.write(str(word)+" ")
    outfile.close()
    endTime=time.clock()
    print("Time Spent Saving to vectorInput.txt= "+ str(endTime-midTime)+" sec.")
endTime=time.clock()
print("Total Time Spent = "+ str(endTime-startTime)+" sec.")
   

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Start Processing .srt files...
Operation = 3


Loading model cost 0.572 seconds.
Prefix dict has been built succesfully.


Total Moive number = 10
Time Spent Processing Srt. = 1.3462020000000001 sec.
Total Time Spent = 1.346442 sec.


In [3]:
#---------------load preprocessed 300_selected words for each type of movies------------------------
#save 300 words to list
import pickle
print("Start loading 300_selected...")
print("Operation = "+str(operationType))
startTime=time.clock()
words300In=open("300_Selected.pickle",'rb')
wordin=pickle.load(words300In)
movieType=['action','adventure','comedy','horror','romance','sci_fi']
keywordList={}
for tp in movieType:
    tmpKeys=[]
    for k,v in wordin[tp].items():
        tmpKeys.append(k)
    keywordList.update({tp:tmpKeys})
print("Done loading! Words saved to variable: keywordList")
endTime=time.clock()   
print("Total Time Spent = "+ str(endTime-startTime)+" sec.")

Start loading 300_selected...
Operation = 3
Done loading! Words saved to variable: keywordList
Total Time Spent = 0.002857999999999805 sec.


In [4]:
#--------------------------------------Jaccard similarity-------------------------------
#----------------------------------Use Jaccard to find the type of each input movie-----
#----------------------------------Only for operation 2 & 3 ----------------------------
from __future__ import division
import string
import math

def jaccard_similarity(query, document):#---the bigger the score, the more similar
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

startTime=time.clock()
print("Start calculating similarity to types...")
movieType=['action','adventure','comedy','horror','romance','sci_fi']
scores={}
FoundTypes=[]
cnt=0
for movie in AllInput:
    cnt+=1
    for ty in movieType:
        score=jaccard_similarity(movie,keywordList[ty])
        scores.update({ty:score})
        #print("Type: "+ty+"\t - Score: \t"+ str(score))
    sortedD=sorted(scores.items(), key=lambda x: x[1], reverse=True)
    foundType=sortedD[0][0]
    #print("第:"+str(cnt)+"部電影 -Type Found! :"+foundType)
    FoundTypes.append(foundType)
print("Operation = "+str(operationType))
print("Total movie processed = " + str(cnt))
endTime=time.clock()   
print("Total Time Spent = "+ str(endTime-startTime)+" sec.")


Start calculating similarity to types...
Operation = 3
Total movie processed = 10
Total Time Spent = 0.031115000000000226 sec.


In [5]:
#--------------------------------------find most similar movie----------------------------------------
#--------------------------------used previously saved AllInput variable------------------------------
AllsubsIn=open("AllSubWordsSegmented.pickle",'rb')
WordsofMovies=pickle.load(AllsubsIn)
movieSimilarity={}
topmovies={}
tmpsim=[]
testandGenerate=[]

startTime=time.clock()
print("Operation = "+str(operationType))
print("Start calculating similarity to movies...")
for i in range(len(AllInput)):
    tmpsim=[]
    movieSimilarity={}
    for k,v in WordsofMovies[FoundTypes[i]].items():
        tmpScore=jaccard_similarity(AllInput[i],v)
        movieSimilarity.update({k:tmpScore})
    sortedSimilarity=sorted(movieSimilarity.items(), key=lambda x: x[1], reverse=True)
    tmpsim.append(FoundTypes[i])
    if operationType==1:                                    #for training
        tmpsim.append(sortedSimilarity[0:5])                #change the length to get more input for S2s training
        sortedSimilarity=[]
        topmovies.update({str(i+1):tmpsim})
    elif operationType==2 or operationType==3:             #for test or generate
        tmpsim.append(sortedSimilarity[0])                 #get the most similar movie and use the overlapping 
        sortedSimilarity=[]                                #words as the input for S2s
        topmovies.update({str(i+1):tmpsim})
#print(topmovies)
AllsubsIn.close()
endTime=time.clock()   
print("Total Time Spent = "+ str(endTime-startTime)+" sec.")

Operation = 3
Start calculating similarity to movies...
Total Time Spent = 1.3786129999999996 sec.


In [6]:
import random
import pickle
#--------------------------------get random 10 key words from the top similar movies---------------------
MovieWordsRef=open('AllSubWordsSegmented.pickle','rb')
MovieWordsload=pickle.load(MovieWordsRef)
#-----------operation 1----------------------------
startTime=time.clock()
if operationType==1:                                            #for getting training data
    print("Starting input word selection; for Op 1...")
    S2sInput={}                                                 #This gets the words of each most similar movies
    tmpWord=[]                                                  #that overlaps with the 300_selected words    
    tmpallWords=[]
    for k,v in topmovies.items():
        tmpallWords=[]
        MoType=v[0]
        for i in range(len(v[1])):
            mv=v[1][i][0]
            tmpWord=[]
            for wd in keywordList[MoType]:
                if wd in MovieWordsload[MoType][mv]:
                    tmpWord.append(wd)
            tmpallWords.append(tmpWord)
        S2sInput.update({k:tmpallWords})                      #---got similar words between 300 and the movies
           
    totalRandom={}
    print("Start 1st Random Process...")
    cnt=0
    print("Preparing long training input")
    for k,v in S2sInput.items():                              #---get words out of those words
        tempoList=[]
        cnt+=1
        for corpus in v:
            listR=random.sample(corpus, longTrainingInput)    #get 5 words for input
            tempoList.append(listR)
        totalRandom.update({k:tempoList})
    print("Processed movies = " + str(cnt))
    secTotalRandom={}
    print("Start 2nd Random Process...")
    cnt=0
    print("Preparing short training input")
    for k,v in S2sInput.items():
        tempoList=[]
        cnt+=1
        for corpus in v:
            listR=random.sample(corpus, shortTrainingInput)     #get 3 wotds for input
            tempoList.append(listR)
        secTotalRandom.update({k:tempoList})
    endTime=time.clock()   
    print("Processed movies = " + str(cnt))
    print("Total Time Spent preparing training data = "+ str(endTime-startTime)+" sec.")
#-----------operation 2 or 3----------------------------
elif operationType==2 or operationType==3:                   #for test or generation data
    print("Starting input word selection...")
    print("Operation = " +str(operationType))
    S2sInput={}
    tmpWord=[]
    tmpallWords=[]
    for k,v in topmovies.items():
        tmpallWords=[]
        MoType=v[0]
        #for i in range(len(v[1])):
        mv=v[1][0]
        tmpWord=[]
        for wd in keywordList[MoType]:
            if wd in MovieWordsload[MoType][mv]:
                tmpWord.append(wd)
        tmpallWords.append(tmpWord)
        S2sInput.update({k:tmpallWords})                    #---got similar words between 300 and the movies

    totalRandom={}
    print("Start Random Process")
    cnt=0
    for k,v in S2sInput.items():
        tempoList=[]
        cnt+=1
        for corpus in v:
            listR=random.sample(corpus, generateInput)      #---get 2 words for input
            tempoList.append(listR)
        totalRandom.update({k:tempoList})
    endTime=time.clock()   
    print("Processed movies = " + str(cnt))
    print("Total Time Spent preparing generation data = "+ str(endTime-startTime)+" sec.")

Starting input word selection...
Operation = 3
Start Random Process
Processed movies = 10
Total Time Spent preparing generation data = 0.32951200000000025 sec.


In [7]:
#-------------------------Out to Sequence2Sequence use titleDict to find title in Chinese
#This part is more for training the S2s, real execution of S2s generation needs no target
startTime=time.clock()
if operationType==1:
    print("Operation "+str(operationType)+" Starts...")
    preS2s={}
    preS2s2={}
    target_line_dict={}
    target_line_dict2={}
    S2sInput=open("S2sTrainingInput.txt",'w',encoding="utf-8")
    for i in range(len(totalRandom)):
        preS2s.update({movieOrder[i]:totalRandom[str(i+1)]})
    for k,v in preS2s.items():
        target_line_dict.update({titleDict[k]:v})
    #---------------------------
    for i in range(len(secTotalRandom)):
        preS2s2.update({movieOrder[i]:secTotalRandom[str(i+1)]})
    for k,v in preS2s2.items():
        target_line_dict2.update({titleDict[k]:v})
    #--------------------------
    for k,v in target_line_dict.items():
        tmptitle=[]
        for w in k:
            if w == ":" or w == "2" or w =="3" or w =="4" or w =="！" or w =="?" or w == "1" or w == "5" or w == "7" or w == "0" or w == "8" or w == "9" or w == "6" or w == "X":
                pass
            else:
                tmptitle.append(w)
        k_in="".join(tmptitle)  
        seg_k = jieba.cut(k_in,cut_all=False)
        mtitle=" ".join(seg_k)
        for l in v:
           #print(" ".join(l)+" | "+mtitle)
            S2sInput.write(" ".join(l)+" | "+mtitle+"\n")
    #----------------------------------------------
    for k,v in target_line_dict2.items():
        tmptitle=[]
        for w in k:
            if w == ":" or w == "2" or w =="3" or w =="4" or w =="！" or w =="?" or w == "1" or w == "5" or w == "7" or w == "0" or w == "8" or w == "9" or w == "6" or w == "X":
                pass
            else:
                tmptitle.append(w)
        k_in="".join(tmptitle)  
        seg_k = jieba.cut(k_in,cut_all=False)
        mtitle=" ".join(seg_k)
        mtitle=mtitle.split()
        mtitle=str(mtitle[0])
        for l in v:
            #print(" ".join(l)+" | "+mtitle)
            S2sInput.write(" ".join(l)+" | "+mtitle+"\n")
    S2sInput.close()
    print("Operation "+str(operationType)+" Done...")
#--------------------------------------------------------Op 2 or 3-----------------------------
elif operationType==2 or operationType==3:
    print("Operation "+str(operationType)+" Starts...")
    S2sInput=open("S2sTestInput.txt",'w',encoding="utf-8")
    preS2s={}
    target_line_dict={}
    for i in range(len(totalRandom)):
        preS2s.update({movieOrder[i]:totalRandom[str(i+1)]})
    for k,v in preS2s.items():
        target_line_dict.update({k:v})
    for k,v in target_line_dict.items():
        for l in v:
            #print(" ".join(l)+" |")
            S2sInput.write(" ".join(l)+" |\n")
    S2sInput.close()
    print("Operation "+str(operationType)+" Done...")
#-----------to s2s input done
endTime=time.clock()
print("Total Time Spent outputing data = "+ str(endTime-startTime)+" sec.")

Operation 3 Starts...
Operation 3 Done...
Total Time Spent outputing data = 0.0011100000000006105 sec.


In [8]:
#prepare vector.bin
#password=mouse54321
import getpass
import os
startTime=time.clock()
if operationType==1:
    password = getpass.getpass()
    command="sudo -S ./ChatBotCourse-master/word2vec/word2vec -train ./vectorInput.txt -output vectors.bin -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-5 -threads 20 -binary 1 -iter 15"
    os.system('echo %s | %s' % (password, command))
elif operationType==2 or operationType==3:
    pass

print("Vector preparation done!")
endTime=time.clock()
print("Total Time Spent = "+ str(endTime-startTime)+" sec.")

Vector preparation done!
Total Time Spent = 0.00028100000000019776 sec.


In [9]:
#Run generator or train
startTime=time.clock()
if operationType==1:
    !python "ChatBotCourse-master/chatbotv2/my_seq2seq_v5.py" train
if operationType==2 or operationType==3:
    !python "ChatBotCourse-master/chatbotv2/my_seq2seq_v5.py" test "ChatBotCourse-master/chatbotv2/S2sTestInput.txt"
endtime=time.clock()
print("Time Spent = "+ str(endtime-startTime)+" sec.")

/home/ymc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-01-31 13:32:44.673253: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
begin load vectors
words = 31948
size = 200
load vectors finish

predict answer

干活 0.2078247447905392 0.9414782681835098
启示录 0.3427040660677401 0.41106948246021013
有钱 0.2816311909595598 0.15946990456335255
太久 0.2708094404751138 0.06386107220210724
拖 0.2553119397095464 0.028170949338043722
未来 0.3075988194645857 0.011873412584381095

predict answer

失去 0.2563361124987004 0.8685160350530305
启示录 0.4957927498887571 0.5037788394813076
扩散 0.3990382133461168 0.17135288261349135
令 0.356470436143082

In [10]:
#--------get output from s2s and generate title

import jieba
import jieba.posseg
import itertools
import random
import copy
# r = ("r")
# v = ("s v n")
# a = ("a")
# t = ("t")
# s = ("s v n")
# n = ("s v n")
#-----collect have the most common combination of movie titles
startTime=time.clock()
collect = [['nrt'],['nr','n'],['vn'],['ns','vn'],['a','ns'],['a','n'],["s","v","n"],['v','c','ns'],['n','n'],['v'],['n'],["r","a","v","n"],['sv','ns'],['ns'],['a']]
#'r':0,'n':0,'v':0,'a':0,'t':0,'s':0

f = open('output.txt','r',encoding = 'utf-8')
movie_list = []                                   #save output from s2s to list and get the post_tag of the words
for text in f:
    
    text = text.strip('\n')
    tmp = text.split(" ")
    #print(tmp)
    for i in range(0,len(tmp)):
        if(tmp[i] != ''):
            movie_list.append(tmp[i])
    
print("All title saved to list for Post_Tag!")
print("Post_tagging Starts...")
d =[]

word_list = []
pos = []

for i in range(len(movie_list)):                  #start pos_tagging
    seg = jieba.posseg.cut(movie_list[i])
    aa = []
    for j in seg:
        if(movie_list[i] == '之'):
            j.flag = 'c'
        aa.append(j.word)
        aa.append(j.flag)
        d.append(j.flag)
        word_list.append(j.word)
        break
print("Post_tagging Done!")


endtime=time.clock()
print("Time Spent = "+ str(endtime-startTime)+" sec.")

All title saved to list for Post_Tag!
Post_tagging Starts...
Post_tagging Done!
Time Spent = 0.002813999999999872 sec.


In [12]:
#---choose best combination
from hanziconv import HanziConv
opp  = open("answer.txt",'w',encoding ='utf-8')
combinations = []
#print(d)
for i in range(int(len(word_list)/6)):
    tagging = {}
    clock = 0
    for x in word_list[i*6:i*6+6]:
        temp = {}
        #print(d[i*6+x])
        #print(x)
        #print(x)
        if d[i*6+clock] not in tagging:
            t = []
            t.append(x)
            temp['count'] = 1 
            temp['word'] = t
            tagging[d[i*6+clock]] = temp
        else:
            tagging[d[i*6+clock]]['count'] += 1 
            tagging[d[i*6+clock]]['word'].append(x)
        clock +=1
            
    #print(tagging)
   
    tag = copy.deepcopy(tagging)
    for x in collect:
        answer = []
        index = len(x)
        #print(" -- - - - - - - -  - -")
        for y in x:
            #print(y)
            if(y in tag and tag[y]['count']>0):
                index -= 1
                
                num = random.randint(0,100)%(tag[y]['count'])
                #print(tag[y]['count'])
                answer.append(tag[y]['word'][num])
                tag[y]['word'].remove(tag[y]['word'][num])
                tag[y]['count'] -=1
                
            else:
                #print("null")
                tag = copy.deepcopy(tagging)
                #print("++++++:" + str(tagging))
                break
        if(index == 0 ):
            tag = copy.deepcopy(tagging)
            break

    #print(answer)
    write_data =""
    for x in answer:
        write_data += x
    print(write_data)
    opp.write(str(i+1)+'\t' +str(HanziConv.toTraditional(write_data)) +'\n')
    #tmp = itertools.combinations(word_list[i*6:i*6+6], 3)
opp.close() 
#     for x in collect:
#         for y in x:
#             for i in tmp:
#                 if(y==d[i[0]]):
#                 print(i)
#                 combinations.append((i[0],i[1]))
#                 combinations.append((i[1],i[0]))

太久启示录
启示录空气
怪物时光
采取
忧郁大脑
饥饿恋人
格列佛流水
王子战役
童贞山
极限人生
